In [ ]:
import numpy as np
import torch
from torch import tensor
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [ ]:
import pickle
with open('/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_1', 'rb') as f:
    text = pickle.load(f,encoding='bytes')

In [ ]:
def load_data(data, batch_id):
    with open('/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_'+str(batch_id),'rb') as f:
        dt = pickle.load(f, encoding='bytes')
    
    data = dt[b'data'].reshape(len(dt[b'data']),3, 32, 32)
    label = dt[b'labels']
    
    return data, label

In [ ]:
d, l = load_data(text, 1)

In [ ]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
def image_show(sample_id):
    image = plt.imshow(d[1].transpose(1,2,0))
    return image

In [ ]:
def explore_data(dt, b_id):
    count_id = np.zeros(10)
    d, l = load_data(dt, b_id)
    for i in (l):
        count_id[i] += 1
    print('batch size: ',len(d))
    for i in range(len(label_names)):
        print(label_names[i],':',count_id[i])
    

In [ ]:
explore_data(text, 2)

In [ ]:
def normalize_min_max(x, x_min, x_max):  
    x = (x- x_min)/(x_max-x_min)
    return x

In [ ]:
def load_set(dt, bid):
    #load the data
    d, l = load_data(dt, bid)
    #split the data in the train and the validation set
    x_train, x_val, y_train, y_val = train_test_split(d, l, test_size = 0.1)
    
    #find the min and the max of the trainin set
    x_min, x_max = np.min(x_train),np.max(x_train)
    
    #normalize the data 
    x_train = normalize_min_max(x_train, x_min, x_max)
    x_val = normalize_min_max(x_val, x_min, x_max)
    
    #from numpy to torch tensors
    x_train, x_val = torch.from_numpy(x_train), torch.from_numpy(x_val)
    
    #train_test_split transforms the labels in list, so I retrasform back in numpy arrays
    y_train, y_val = np.array(y_train), np.array(y_val)
    
    #control the labels are int and transform in torch tensors
    y_train, y_val = y_train.astype(int), y_val.astype(int)
    y_train, y_val = torch.from_numpy(y_train), torch.from_numpy(y_val)
    
    return x_train, y_train, x_val, y_val

In [ ]:
x_train, y_train, x_val, y_val = load_set(text,1)

In [ ]:
x_train.shape,y_val.shape, x_val.shape,y_train.shape

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3,64,3, stride = 1, padding = 1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(64,64,3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64,128,3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128,128,3, stride=1, padding=1)
        self.conv5 =  nn.Conv2d(128,256,3, stride=1, padding=1)
        self.conv6 =  nn.Conv2d(256,256,3, stride=1, padding=1)
        self.conv7 =  nn.Conv2d(256,512,3, stride=1, padding=1)
        self.conv8 =  nn.Conv2d(512,512,3, stride=1, padding=1)
        self.lin1 = nn.Linear(512*2*2, 100)
        self.lin2 = nn.Linear(100,84)
        self.out = nn.Linear(84,10)
    
    def forward(self, x):
        x = (F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = (F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = (F.relu(self.conv5(x)))
        x = self.pool(F.relu(self.conv6(x)))
        x = (F.relu(self.conv7(x)))
        x = self.pool(F.relu(self.conv8(x)))
        x = x.view(-1, 512*2*2)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = (self.out(x))
        
        return x
        
        
        
    

In [ ]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(3, 16, 5)
        self.fc1 = nn.Linear(16 * 14 * 14, 120)
        self.fc3 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = self.fc3(x)
        return x

In [ ]:
import torch.optim as optim

def get_model():
    model = Net()
    loss_func = nn.CrossEntropyLoss()
    return model, loss_func, optim.Adam(model.parameters())

class Learner():
    def __init__(self, model, loss, opt):
        self.model, self.loss, self.opt = model, loss, opt

In [ ]:
def accuracy(x,y): return (torch.argmax(x, axis=1)==y).float().mean()

In [ ]:
def fit(l,x_train,y_train,x_val,y_val, bs=50,epochs=1):
    for epoch in range(epochs):
        l.model.train()
        tot_loss = 0.0
        tot_acc = 0.0
        for i in range(len(x_train)//bs):
            start_ind = i*bs
            end_ind = start_ind + bs

            x = x_train[start_ind:end_ind]
            y = y_train[start_ind:end_ind]

            pred = l.model(x.float())
            loss = l.loss(pred, y)
            acc = accuracy(pred,y)
            
            tot_loss +=loss
            tot_acc += acc
            
            loss.backward()
            
            l.opt.step()
            l.opt.zero_grad()
            
        #print('Epoch: ',epoch,'Loss: ',tot_loss/(len(x_train[0:200])//bs ),'Accuracy: ',tot_acc/(len(x_train[0:200])//bs ))
            
        l.model.eval()
        with torch.no_grad():
            tot_loss = 0.0
            tot_acc = 0.0
            for i in range(len(x_val)//bs):
                start_ind = i * bs
                end_ind = start_ind + bs
                
                x_v = x_val[start_ind:end_ind]
                y_v = y_val[start_ind:end_ind]
                
                pred = l.model(x_v.float())
                loss_t = l.loss(pred, y_v)
                acc_t = accuracy(pred, y_v)
                
                tot_loss += loss_t
                tot_acc += acc_t
        print('Epoch: ',epoch,'Loss: ',tot_loss/(len(x_val)//bs ),'Accuracy: ',tot_acc/(len(x_val)//bs ))
        
        

In [ ]:
lrn = Learner(*get_model())

In [ ]:
for i in range(1,5):
    x_train,y_train,x_val,y_val = load_set(text,i)
    print(x_train.shape,x_val.shape,y_train.shape,y_val.shape)
    fit(lrn,x_train,y_train,x_val,y_val, epochs=20)